Test Phase4_3d file

In [12]:
# install anaconda: https://www.anaconda.com/products/individual
# if use Visual Studio Code, install python extension and select the python interpreter that includes anaconda
# install micropip: https://pypi.org/project/micropip/. You can also run anaconda prompt and type "pip install micropip"
import numpy as np
import micropip
await micropip.install("scipy")

In [13]:
# from Phase4_trace_3d import Phase4_trace_3d
from generateTestPositions import generateTestPositions
import generateTestPositions
from generateTestPositions import get_velocity_function
from generateTestPositions import generateTestPositions
from generateTestPositions import get_initial_position
# Cell 4
# Testing with dummy data
NumOfDataPoints = 1
clusterness = 0.1 # smaller number the more clustered
# Input conditions
initial_positions = []
v = []
# for i in range(1):
#     [x,y,z] = get_initial_position(i)
#     initial_positions.append([x, y, z])
#     vel = get_velocity_function(i)
#     v.append(vel)
initial_positions = np.zeros((NumOfDataPoints,3))
initial_positions[0] = get_initial_position(1)
v.append(lambda t: [np.sin(t), np.cos(t), 1])
noise = 1e-3
theta_degrees = 1.8
rev = 2  # revolutions of camera for the entire process
NOS = int(rev * 360 / theta_degrees)
NOS = 200
NOS_per_section = 160  # must be larger than 5 to satisfy equations
camera_speed = 0.5  # in Hz or revolution per second
SRD = 20  # mm, Source-Reference Distance
RDD = 480  # mm, Reference-Detector (screen) Distance
radius = 1

pixelResolution = 0.172  # every pixel is equal to mm
method = 'acceleration'
dataPiling = 'overlap'

# Auto-calculations of the rest of the parameters derived from the setting above
delta_T = camera_speed * theta_degrees / 360
shots_per_second = 1 / delta_T

# Define the velocity function
# v = lambda t: [0.9 * np.sin(t), 0.9 * np.cos(t), 1]

# Pack conditions into a list
conditions = [noise, delta_T, NOS, theta_degrees, NOS_per_section, SRD, RDD,method,dataPiling]
print(conditions)

xz_proj = np.zeros((NOS, NumOfDataPoints*2))
real_positions = np.zeros((NOS, NumOfDataPoints*3))
# Generate test positions
for i in range(NumOfDataPoints):
    vel = v[i]
    xz_proj[:,i*2:i*2+2], real_positions[:,i*3:i*3+3]= generateTestPositions(vel, initial_positions[i], conditions)

[0.001, 0.0025, 200, 1.8, 160, 20, 480, 'acceleration', 'overlap']


In [14]:
# Print the first few rows of the generated data
# print("xz_proj:")
# print(xz_proj[:])
# print("real_positions:")
# print(real_positions[:])

In [15]:
import pandas as pd
import os

def rename_files_replace_space(directory_path):
    # Get a list of all files in the directory
    filenames = os.listdir(directory_path)
    
    for filename in filenames:
        # Replace spaces with underscores
        new_filename = filename.replace('_', '')
        
        # Construct the full old and new file paths
        old_filepath = os.path.join(directory_path, filename)
        new_filepath = os.path.join(directory_path, new_filename)
        
        # Rename the file
        os.rename(old_filepath, new_filepath)

In [16]:
# Import the function directly from the module
from Phase4_trace_3d import Phase4_trace_3d
from smooth_points import smooth_points
from particle_path_finder import ParticlePathFinder as pf
# from particle_path_finder import OriginalPathKeeper as opk
import random
import numpy as np
import pandas as pd
import os


# # read data from .csv file
# input_data = pd.read_csv('particle_tracking_input.csv')

# columnIndex = 0

theta_degrees = 1.8
# # Ai recognition caller
alpha = np.radians(theta_degrees)  # Example rotation angle in radians

learning_rate_2D = 0.3
motion_randomness = 3
learning_rate_3D = 0.3

# define different learning rates for investigating different sorting and smoothing model
rates_conditions = [learning_rate_2D, motion_randomness, learning_rate_3D, NOS_per_section]

path_finder = pf(alpha,rates_conditions,conditions)
# original_path_keeper = opk(alpha)
# iterate through each row of the input data as a snapshot, and put the snapshot into the particle path finder
# rename_files_replace_space("Data_3particles")

# print(os.listdir("Data_3particles"))
folderName = "5-Nov-23"
sorted_filenames = sorted(os.listdir(folderName), key=lambda x: int(x.split('Shot')[1].split('.csv')[0]))
print(sorted_filenames)
k = 0
for file in sorted_filenames:
    
    if file.endswith(".csv"):
        filename = os.path.join(folderName, file)
        input_data = pd.read_csv(filename, header=None)
        input_data = np.array(np.transpose(input_data))
        values =  input_data[0]
        print("values:", values)
        print("read file: ", filename)

        offset = [243.5, 97.5]
        paired_values = []
        i = 0
        for j in range(len(values)//2):
            
            inputList = (values[i:i+2]- offset)*pixelResolution
           
            # input format, list of tuple of two elements (x,y)
            paired_values.append(inputList)

            # scambled_values = random.shuffle(paired_values.copy())
            # print("paired values:", paired_values)
            
            i += 2

        # print("paired values:", paired_values)
        paired_values.append(xz_proj[k])
        path_finder.append(paired_values)
        k += 1
    

# Assuming path_finder.get_particle_data() returns your data as a dictionary
sorted_particle_data = path_finder.get_particle_data()



['Shot0.csv', 'Shot1.csv', 'Shot2.csv', 'Shot3.csv', 'Shot4.csv', 'Shot5.csv', 'Shot6.csv', 'Shot7.csv', 'Shot8.csv', 'Shot9.csv', 'Shot10.csv', 'Shot11.csv', 'Shot12.csv', 'Shot13.csv', 'Shot14.csv', 'Shot15.csv', 'Shot16.csv', 'Shot17.csv', 'Shot18.csv', 'Shot19.csv', 'Shot20.csv', 'Shot21.csv', 'Shot22.csv', 'Shot23.csv', 'Shot24.csv', 'Shot25.csv', 'Shot26.csv', 'Shot27.csv', 'Shot28.csv', 'Shot29.csv', 'Shot30.csv', 'Shot31.csv', 'Shot32.csv', 'Shot33.csv', 'Shot34.csv', 'Shot35.csv', 'Shot36.csv', 'Shot37.csv', 'Shot38.csv', 'Shot39.csv', 'Shot40.csv', 'Shot41.csv', 'Shot42.csv', 'Shot43.csv', 'Shot44.csv', 'Shot45.csv', 'Shot46.csv', 'Shot47.csv', 'Shot48.csv', 'Shot49.csv', 'Shot50.csv', 'Shot51.csv', 'Shot52.csv', 'Shot53.csv', 'Shot54.csv', 'Shot55.csv', 'Shot56.csv', 'Shot57.csv', 'Shot58.csv', 'Shot59.csv', 'Shot60.csv', 'Shot61.csv', 'Shot62.csv', 'Shot63.csv', 'Shot64.csv', 'Shot65.csv', 'Shot66.csv', 'Shot67.csv', 'Shot68.csv', 'Shot69.csv', 'Shot70.csv', 'Shot71.csv', '

In [17]:
print("sorted_positions: ", sorted_particle_data)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(sorted_particle_data)

# Select only the first row of the DataFrame
first_row_df = df.iloc[0,0]

first_row_df = np.transpose(np.transpose(first_row_df))
print(first_row_df)
print(first_row_df.shape)

df_to_save = pd.DataFrame(first_row_df)

# Write the DataFrame to an Excel file
df_to_save.to_excel('output.xlsx', index=False)

sorted_positions:  {0: {'coords': [array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([-0.086,  0.   ]), array([0.086, 0.   ]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0.086, 0.   ]), array([0., 0.]), array([0.086, 0.   ]), array([0.086, 0.   ]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0.086, 0.   ]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0., 0.]), array([0.

In [18]:
NumOfDataPoints = len(sorted_particle_data)
print("NumOfDataPoints: ", NumOfDataPoints)
estimated_positions = np.zeros((NOS,3*NumOfDataPoints))
for i in range(NumOfDataPoints):
    estimated_positions_single = Phase4_trace_3d(conditions, np.array(sorted_particle_data[i]['coords']))
    estimated_positions_single = smooth_points(estimated_positions_single, 'sg',NOS_per_section)

    estimated_positions[:,i*3:i*3+3] = estimated_positions_single



NumOfDataPoints:  4
NOS:  200
NOS_per_Section:  160
iterations:  0
The runtime of generateEstimatedPositions is 1.1253488063812256 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  1
iterations:  2
The runtime of generateEstimatedPositions is 0.5202953815460205 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  3
The runtime of generateEstimatedPositions is 0.9902491569519043 seconds.
The runtime of adding 3D positions is 0.0010082721710205078 seconds.
iterations:  4
The runtime of generateEstimatedPositions is 0.5708067417144775 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  5
The runtime of generateEstimatedPositions is 0.5019826889038086 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  6
The runtime of generateEstimatedPositions is 0.8389701843261719 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  7
The runtime of generateEstimatedPositions is 0.6352424621582031 se

In [19]:
# extra synthetic data points (bypassing sorting)
estimated_positions_single = smooth_points(Phase4_trace_3d(conditions, xz_proj), 'sg',NOS_per_section)


NOS:  200
NOS_per_Section:  160
iterations:  0
The runtime of generateEstimatedPositions is 0.5223913192749023 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  1
iterations:  2
The runtime of generateEstimatedPositions is 0.5222160816192627 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  3
The runtime of generateEstimatedPositions is 0.7824051380157471 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  4
The runtime of generateEstimatedPositions is 0.5903491973876953 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  5
The runtime of generateEstimatedPositions is 0.508899450302124 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  6
The runtime of generateEstimatedPositions is 1.0316901206970215 seconds.
The runtime of adding 3D positions is 0.0 seconds.
iterations:  7
The runtime of generateEstimatedPositions is 1.061730146408081 seconds.
The runtime of adding 3D position

In [20]:
print("size of estimated_positions_single: ", len(estimated_positions_single))
print("size of estimated_positions: ", len(estimated_positions))
estimated_positions_new = np.column_stack((estimated_positions, estimated_positions_single))
NumOfDataPoints += 1
print("size of estimated_positions: ", len(estimated_positions_new))
print('estimated positions: \n',estimated_positions_new)

# print('real positions: \n',real_positions)

size of estimated_positions_single:  200
size of estimated_positions:  200
size of estimated_positions:  200
estimated positions: 
 [[ 8.07857771e-03 -4.88672683e-03  2.89721284e-03 ...  1.94983383e-01
  -7.89365382e-03  3.67123963e-06]
 [ 8.00334429e-03 -4.78121949e-03  2.80835337e-03 ...  1.95222244e-01
  -5.32532970e-03  2.50340829e-03]
 [ 7.92825820e-03 -4.67676464e-03  2.72083964e-03 ...  1.95465410e-01
  -2.75873854e-03  5.00314769e-03]
 ...
 [-5.23853972e-03 -1.32009840e-02  5.68366779e-03 ...  3.16017503e-01
   4.37669153e-01  4.66541883e-01]
 [-5.34155059e-03 -1.34889995e-02  5.71769398e-03 ...  3.16847867e-01
   4.39201001e-01  4.68367932e-01]
 [-5.44533576e-03 -1.37796534e-02  5.75154547e-03 ...  3.17679696e-01
   4.40723757e-01  4.70186826e-01]]


In [21]:
# plotting

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random
def number_to_binary_list(number):
    binary_str = bin(number)[2:]  # Convert to binary and remove the '0b' prefix
    binary_str = binary_str.zfill(3)  # Pad with zeros to make sure it has 3 digits
    binary_list = np.array([int(b) for b in binary_str] ) # Convert each binary digit to integer
    return binary_list


def plotting_single(positions, particle_id,ax):
    NOS, _ = positions.shape
    particle_id=int(particle_id)+1
    if particle_id<7 and particle_id>=0:
        col=number_to_binary_list(int(particle_id))
    else:
        col=np.array([random.uniform(0.6, 1) for _ in range(3)])
        print(col)

    
    # Plot the curve with gradually changing color
    for i in range(NOS - 1):
        color_brightness = (i/(NOS-1))*col
        
        x1, y1, z1 = positions[i]
        x2, y2, z2 = positions[i+1]
        
        # Draw a line segment with the computed color
        ax.plot([x1, x2], [y1, y2], [z1, z2], color=color_brightness)
    ax.plot([x1, x2], [y1, y2], [z1, z2], color=color_brightness,label='particle'+str(particle_id-1))
    # Label the axes
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.legend()
    

In [22]:
import matplotlib
print(matplotlib.get_backend())
import matplotlib.pyplot as plt
matplotlib.use('TkAgg')  # or another interactive backend


# Create the figure and axes
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
for i in range(NumOfDataPoints):
    plotting_single(estimated_positions_new[:,i*3:i*3+3],i,ax)

    
plt.show()


TkAgg
